In [ ]:
# G. Hayes 2025
# This is script is used to load in the preprocessed TCD and MRI ramp data and fit it with linear and non-linear (4- and 2-parameter) models using least squares regression and Bayesian fitting for the analysis presented in:
# G. Hayes, S. Sparks, J. Pinto, and D. P. Bulte, "Models of Cerebrovascular Reactivity in BOLD-fMRI and Transcranial Doppler Ultrasound"

# To run a batch of this script, use the "mni_mri_tcd_analysis" script.
# To run the preprocessing of the TCD data, use the "1_tcd_ramp_preprocessing.ipynb" script.
# To run the preprocessing of the MRI data, use the "2_mri_co2_preprocessing.ipynb" script.

# To run the fitting of the data after the preprocessing, use the "3_mni_mri_tcd_analysis.ipynb" script.

# Updated this script for your purposes, notably:
# - the data folder name
# - the data file name
# - the output file name
# - alter parameters that may differ for your data


In [ ]:
# This script will load in the preprocessed mri+petco2 data and well as the tcd+petco2 data and plot them

import numpy as np
import os,sys
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import resample
import scipy
from scipy.signal import find_peaks
from scipy import stats
from scipy import optimize
from scipy.optimize import curve_fit
from scipy.stats.distributions import t
import seaborn as sns
import copy
import csv
sys.path.append('..')
import ramp_cvr_functions as cvr_func
import time

from BayesicFitting import PolynomialModel
from BayesicFitting import LogisticModel
from BayesicFitting import PowerLawModel
from BayesicFitting import GaussPrior
from BayesicFitting import NestedSampler
from BayesicFitting import formatter as fmt

save_fig = True
save_log = True

region_name= "r6r"

In [ ]:
#PARAMETERS FOR BATCH ANALYSIS
# TCD data
tcd_local= ""
mca_filepath= ""
petco2_filepath= ""
# MRI data
mri_local= ""
func_filepath= ""
mrpetco2_filepath= ""
sub= "" #mr subject number
sub_tcd= "" #tcd subject number
mca_peak_min_distance= ""
mca_peak_prominence= ""
mca_start_peak_search= ""

In [ ]:
# IF NOT USING THE BATCH SCRIPT, UNCOMMENT THE FIRST PARAGRAPH, UPDATE THE PARAMETERS, AND RUN FOR EACH SUBJECT

# # # MR subject 003, TCD subject 021
# # TCD data
# tcd_local = 'path/to/tcd/data/'
# mca_filepath = 'sub-021/ses-01/sub-021_ses-01_YYYYMMDD_task_task-ramp_MCAvmean.csv'
# petco2_filepath = 'sub-021/ses-01/sub-021_ses-01_YYYYMMDD_task_task-ramp_petco2.csv'
# # MRI data
# mri_local = 'path/to/mri/data/'
# func_filepath = 'mr-003_func_final_' + region_name + '.csv'
# mrpetco2_filepath = 'mr-003_petco2_final_' + region_name + '.csv'
# sub = 'MR-003'
# sub_tcd = 'SUB-021'
# mca_peak_min_distance = 110
# mca_peak_prominence = 50
# mca_start_peak_search = 110

# # # # MR subject 004, TCD subject 022 
# # TCD data
# tcd_local = 'path/to/tcd/data/'
# mca_filepath = 'sub-022/ses-01/sub-022_ses-01_YYYYMMDD_task_task-ramp_MCAvmean.csv'
# petco2_filepath = 'sub-022/ses-01/sub-022_ses-01_YYYYMMDD_task_task-ramp_petco2.csv'
# # MRI data
# mri_local = 'path/to/mri/data/'
# func_filepath = 'mr-004_func_final_' + region_name + '.csv'
# mrpetco2_filepath = 'mr-004_petco2_final_' + region_name + '.csv'
# sub = 'MR-004'
# sub_tcd = 'SUB-022'
# mca_peak_min_distance = 110
# mca_peak_prominence = 50
# mca_start_peak_search = 110


In [ ]:
outdir = 'path/to/output/'
sub_outdir = outdir + sub + '/' 
print('sub_outdir:', sub_outdir)
# make a directory for the processed data if it doesn't already exist
if not os.path.exists(sub_outdir):
    os.makedirs(sub_outdir)

# Create log directory with date and time
#timestr = time.strftime("%Y%m%d-%H%M")
timestr = time.strftime("%Y%m%d")
outdir_logs = outdir + 'logs_' + timestr + '/'
print('outdir_logs:', outdir_logs)

if save_log:
    if not os.path.exists(outdir_logs):
        os.makedirs(outdir_logs)

In [ ]:
# Preprocessed TCD data


# Load in the MCAvmean and petco2 data by any filename ending in _MCAvmean.csv and _petco2.csv
mca = pd.read_csv(tcd_local + mca_filepath)
petco2 = pd.read_csv(tcd_local + petco2_filepath)
# Remove the extra columns
mca = mca.drop(columns=['Unnamed: 0'])
petco2 = petco2.drop(columns=['Unnamed: 0'])

# print the shape of the data
print(mca.shape)
print(petco2.shape)
# print the headers of the data
print(mca.head())
print(petco2.head())
# print the number of nan values in the data
print('Number of NaN values in MCAvmean data:',mca.isnull().sum())
print('Number of NaN values in PETCO2 data:',petco2.isnull().sum())
print('-- Removing NaN values --')
# find indices of nan values
mca_nan_indices = np.where(mca.isnull().values)
petco2_nan_indices = np.where(petco2.isnull().values)
nan_indices = np.unique(np.concatenate((mca_nan_indices[0],petco2_nan_indices[0])))

# remove the nan indices from both datasets
mca = mca.drop(nan_indices)
petco2 = petco2.drop(nan_indices)
print('New shape of MCAvmean data:',mca.shape)
print('New shape of PETCO2 data:',petco2.shape)

# Resample the data to a given number of data points
print('-- Downsampling values --')
tcd_num_samples = 769
mca = resample(mca, tcd_num_samples)
petco2 = resample(petco2, tcd_num_samples)
# redefine mca and petco2 as dataframes
mca = pd.DataFrame(mca)
petco2 = pd.DataFrame(petco2)


# reindex the data
# mca = mca.reset_index(drop=True)
# petco2 = petco2.reset_index(drop=True)
print('New shape of MCAvmean data:',mca.shape)
print('New shape of PETCO2 data:',petco2.shape)


# plot MCAvmean and PETCO2 data
fig_tcd_ready = plt.figure(figsize=(10,3))

plt.plot(mca,label='MCAvmean')
plt.xlabel('Time')
plt.ylabel('MCAvmean (cm/s)', color='b')
# color the y-axis blue
plt.tick_params(axis='y',labelcolor='b')
# Add another axis to plot petco2
ax2 = plt.twinx()
ax2.plot(petco2,'r', label='PETCO2')
ax2.set_ylabel('PETCO2 (mmHg)',color='r')
ax2.tick_params(axis='y',labelcolor='r')

In [ ]:
# Preprocessed MRI data


# Load in the functional and petco2 data by any filename ending in _func_final.csv and _petco2_final.csv
func = pd.read_csv(mri_local + func_filepath)
mrpetco2 = pd.read_csv(mri_local + mrpetco2_filepath)

# print the shape of the data
print(func.shape)
print(mrpetco2.shape)

# plot MCAvmean and PETCO2 data
fig_mri_ready = plt.figure(figsize=(10,3))

plt.plot(func,label='MCAvmean')
plt.xlabel('Time')
plt.ylabel('BOLD (%)', color='b')
# color the y-axis blue
plt.tick_params(axis='y',labelcolor='b')
# Add another axis to plot petco2
ax2 = plt.twinx()
ax2.plot(mrpetco2,'r', label='PETCO2')
ax2.set_ylabel('PETCO2 (mmHg)',color='r')
ax2.tick_params(axis='y',labelcolor='r')


In [ ]:
# Plot mca as a function of petco2
fig_mca_petco2 = plt.figure(figsize=(10,5))
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(petco2, mca, s=4)
plt.xlabel('PETCO2 (mmHg)')
plt.ylabel('MCAvmean (cm/s)')
plt.title('MCAvmean vs PETCO2 -AND- BOLD vs PETCO2')

print(type(mrpetco2))
print(type(func))
print(type(petco2))
print(type(mca))

# plot func as a function of petco2 on the same plot with a different y-axis on the right
ax2 = plt.twinx()
ax2.scatter(mrpetco2,func,color='r', s=4)
ax2.set_ylabel('BOLD (%)',color='r')
ax2.tick_params(axis='y',labelcolor='r')


plt.show()

In [ ]:
### SMOOTH THE MRI DATA ###
# Smooth the data using a rolling average
# Define the window size for the rolling average
window_size = 15

# Smooth the data within the window
func_smooth = func.rolling(window=window_size,center=False).mean()
mrpetco2_smooth = mrpetco2.rolling(window=window_size,center=False).mean()

print('---SMOOTHED DATA---')
print('FUNC shape:', func_smooth.shape)
print('MRPETCO2 shape:', mrpetco2_smooth.shape)
#print(func_ramp_up_smooth)

# reset the indices
func_smooth = func_smooth.reset_index(drop=True)
mrpetco2_smooth = mrpetco2_smooth.reset_index(drop=True)

# Plot the smoothed data
fig = plt.figure(figsize=(10,3))
plt.plot(func_smooth.iloc[:,0])
plt.xlabel('Time')
plt.ylabel('BOLD (psc)', color='b')
# color the y-axis blue
plt.tick_params(axis='y',labelcolor='b')
# Add another axis to plot petco2
ax2 = plt.twinx()
ax2.plot(mrpetco2_smooth.iloc[:,0],'r')
ax2.set_ylabel('PETCO2 (mmHg)',color='r')
ax2.tick_params(axis='y',labelcolor='r')

In [ ]:
func = func_smooth
mrpetco2 = mrpetco2_smooth

In [ ]:
# Calculate the baseline values for mri and mr-petco2
func_mean = func.mean()
mrpetco2_base = cvr_func.get_baseline_values(mrpetco2, 1, 8)

print('---MRI BASELINE VALUES---')
print('BOLD filtered func average value (PSC):', func_mean)
print('MRPETCO2 baseline value (mmHg)', mrpetco2_base)

In [ ]:
#peak prominence used to be equal to 3 for func ramp_identifier_adv()

print(type(func))
print(func.shape)
# MRI data #
### separate ramp_down and ramp_up data and plot###
func_max_idx, func_min_idx = cvr_func.ramp_identifier_adv(func, peak_min_distance=200, peak_prominence=1, start_search=50)
print('func_max_ind',func_max_idx)
print('func_min_ind',func_min_idx)

mrpetco2_max_idx, mrpetco2_min_idx = cvr_func.ramp_identifier_adv(mrpetco2, peak_min_distance=200, peak_prominence=1, start_search=50)
print('mrpetco2_max_ind',mrpetco2_max_idx)
print('mrpetco2_min_ind',mrpetco2_min_idx)

max_idx = np.zeros(np.shape(func_max_idx))
min_idx = np.zeros(np.shape(func_min_idx))

for i in range(0, len(func_max_idx)):
    print('i=', i)
    if func_max_idx[i] < mrpetco2_max_idx[i]:
        max_idx[i] = func_max_idx[i]
    else:
        max_idx[i] = mrpetco2_max_idx[i]
        print('mrpetco2 max ind is smaller than mca max ind = ', max_idx)

    if func_min_idx[i] > mrpetco2_min_idx[i]:
        min_idx[i] = func_min_idx[i]
    else:
        min_idx[i] = mrpetco2_min_idx[i]

# change max_ind and min_ind to list of integers
max_idx = max_idx.astype(int)
min_idx = min_idx.astype(int)

print('max_idx:', max_idx)
print('min_idx:', min_idx)

func_ramp_up, mrpetco2_ramp_up = cvr_func.ramp_up_segmentor(func, mrpetco2, max_idx, min_idx)
print('func_ramp_up shape:', func_ramp_up.shape)
print('mrpetco2_ramp_up shape:', mrpetco2_ramp_up.shape)

# Increase font size
plt.rcParams.update({'font.size': 20})

# plot the ramp up data
# Define figure
fig = plt.figure(figsize=(10,3))
#set the font to times
plt.rcParams['font.family'] = 'Times New Roman'
plt.plot(func.iloc[:], 'r')
plt.plot(func_ramp_up.iloc[:], 'r', marker='o', linestyle='None')
plt.xlabel('Time (TR)')
plt.ylabel('BOLD (%)', color='r')
# color the y-axis blue
plt.tick_params(axis='y',labelcolor='r')
# Add another axis to plot mrpetco2
ax2 = plt.twinx()
ax2.plot(mrpetco2.iloc[:],'k')
ax2.plot(mrpetco2_ramp_up.iloc[:],'k', marker='o', linestyle='None')
ax2.set_ylabel('PETCO2 (mmHg)',color='k')
ax2.tick_params(axis='y',labelcolor='k')


print('func shape:',func.shape)
print('func ramp up shape:',func_ramp_up.shape)
print('mrpetco2 shape:',mrpetco2.shape)
print('mrpetco2 ramp up shape:',mrpetco2_ramp_up.shape)

In [ ]:
# save a copy of the figure in as a png in the same folder
if save_fig == True:    
    fig.savefig(sub_outdir + sub + '_BOLDrampup.png')

In [ ]:
# information about the MRPETCO2 and BOLD func data
print('MRI ramp information')
# get the mean of the lowest 3 values of the mrpetco2 data
mrpetco2_min = mrpetco2_ramp_up.iloc[:,0].nsmallest(3)
# get the mean of mrpetco2_min
mrpetco2_min = mrpetco2_min.mean()
# get the highest 3 values of the mrpetco2 data
mrpetco2_max = mrpetco2_ramp_up.iloc[:,0].nlargest(3)
# get the mean of mrpetco2_max
mrpetco2_max = mrpetco2_max.mean()


mrpetco2_diff = mrpetco2_max - mrpetco2_min
print('MRPETCO2 max', mrpetco2_max)
print('MRPETCO2 min', mrpetco2_min)
print('MRPETCO2 diff', mrpetco2_diff)

# get the mean of the lowest 3 values of the mca data
func_min = func_ramp_up.iloc[:,0].nsmallest(3)
# get the mean of mca_min
func_min = func_min.mean()
# get the highest 3 values of the mca data
func_max = func.iloc[:,0].nlargest(3)
# get the mean of mca_max
func_max = func_max.mean()

func_diff = func_max - func_min
print('func max (psc)', func_max)
print('func min (psc)', func_min)
print('func diff (psc)', func_diff)

In [ ]:
### SMOOTH THE TCD DATA ###
# Smooth the data using a rolling average
# Define the window size for the rolling average
window_size = 15

# Smooth the data within the window
mca_smooth = mca.rolling(window=window_size,center=False).mean()
petco2_smooth = petco2.rolling(window=window_size,center=False).mean()

print('---SMOOTHED DATA---')
print('MCAvmean shape:', mca_smooth.shape)
print('PETCO2 shape:', petco2_smooth.shape)
#print(func_ramp_up_smooth)

# reset the indices
mca_smooth = mca_smooth.reset_index(drop=True)
petco2_smooth = petco2_smooth.reset_index(drop=True)

# Plot the smoothed data
fig = plt.figure(figsize=(10,3))
plt.plot(mca_smooth.iloc[:,0])
plt.xlabel('Time')
plt.ylabel('MCAvmean (cm/s)', color='b')
# color the y-axis blue
plt.tick_params(axis='y',labelcolor='b')
# Add another axis to plot petco2
ax2 = plt.twinx()
ax2.plot(petco2_smooth.iloc[:,0],'r')
ax2.set_ylabel('PETCO2 (mmHg)',color='r')
ax2.tick_params(axis='y',labelcolor='r')


In [ ]:
mca = mca_smooth
petco2 = petco2_smooth

In [ ]:
print(mca.shape)
print(type(mca))
# TCD data #
### separate ramp_down and ramp_up data and plot###
mca_max_idx, mca_min_idx = cvr_func.ramp_identifier_adv(mca, peak_min_distance=mca_peak_min_distance, peak_prominence=mca_peak_prominence, start_search=mca_start_peak_search)
print('mca_max_ind',mca_max_idx)
print('mca_min_ind',mca_min_idx)

petco2_max_idx, petco2_min_idx = cvr_func.ramp_identifier_adv(petco2, peak_min_distance=200, peak_prominence=1, start_search=100)
print('petco2_max_ind',petco2_max_idx)
print('petco2_min_ind',petco2_min_idx)

max_idx = np.zeros(np.shape(mca_max_idx))
min_idx = np.zeros(np.shape(mca_min_idx))

for i in range(0, len(mca_max_idx)):
    print('i=', i)
    if mca_max_idx[i] < petco2_max_idx[i]:
        max_idx[i] = mca_max_idx[i]
    else:
        max_idx[i] = petco2_max_idx[i]
        print('petco2 max ind is smaller than mca max ind = ', max_idx)

    if mca_min_idx[i] > petco2_min_idx[i]:
        min_idx[i] = mca_min_idx[i]
    else:
        min_idx[i] = petco2_min_idx[i]

# change max_ind and min_ind to list of integers
max_idx = max_idx.astype(int)
min_idx = min_idx.astype(int)

print('max_idx:', max_idx)
print('min_idx:', min_idx)

mca_ramp_up, petco2_ramp_up = cvr_func.ramp_up_segmentor(mca, petco2, max_idx, min_idx)
print('mca_ramp_up shape:', mca_ramp_up.shape)
print('petco2_ramp_up shape:', petco2_ramp_up.shape)

# Increase font size
plt.rcParams.update({'font.size': 20})

# plot the ramp up data
# Define figure
fig = plt.figure(figsize=(10,3))
#set the font to times
plt.rcParams['font.family'] = 'Times New Roman'
plt.plot(mca.iloc[:], 'b')
plt.plot(mca_ramp_up.iloc[:], 'b', marker='o', linestyle='None')
plt.xlabel('Time (TR)')
plt.ylabel('MCAvmean (cm/s)', color='b')
# color the y-axis blue
plt.tick_params(axis='y',labelcolor='b')
# Add another axis to plot petco2
ax2 = plt.twinx()
ax2.plot(petco2.iloc[:],'k')
ax2.plot(petco2_ramp_up.iloc[:],'k', marker='o', linestyle='None')
ax2.set_ylabel('PETCO2 (mmHg)',color='k')
ax2.tick_params(axis='y',labelcolor='k')

In [ ]:
if save_fig == True:    
    fig.savefig(sub_outdir + sub + '_MCAvrampup.png')

In [ ]:
# Plot mca as a function of petco2
fig_mca_petco2 = plt.figure(figsize=(10,5))
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(petco2_ramp_up, mca_ramp_up, s=4)
plt.xlabel('PETCO2 (mmHg)')
plt.ylabel('MCAvmean (cm/s)', color='b')
plt.tick_params(axis='y',labelcolor='b')
plt.title('MCAvmean vs PETCO2 AND BOLD vs PETCO2 (ramp up only)')

print(type(mrpetco2))
print(type(func))
print(type(petco2))
print(type(mca))

# plot func as a function of petco2 on the same plot with a different y-axis on the right
ax2 = plt.twinx()
ax2.scatter(mrpetco2_ramp_up,func_ramp_up,color='r', s=4)
ax2.set_ylabel('BOLD (%)',color='r')
ax2.tick_params(axis='y',labelcolor='r')

In [ ]:
if save_fig == True:    
    fig_mca_petco2.savefig(sub_outdir + sub + '_sigmoids.png')

In [ ]:
# Plot mca as a function of petco2
fig_mca_petco2 = plt.figure(figsize=(9,5))
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(petco2_ramp_up, mca_ramp_up, s=4)
plt.xlabel('PETCO$_{2}$ (mmHg)')
plt.ylabel('MCAvmean (cm/s)', color='b')
plt.tick_params(axis='y',labelcolor='b')
plt.title('MCAvmean vs PETCO$_{2}$')

print(type(mrpetco2))
print(type(func))
print(type(petco2))
print(type(mca))

In [ ]:
# Plot mca as a function of petco2
fig_mca_petco2 = plt.figure(figsize=(9,5))
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(mrpetco2_ramp_up,func_ramp_up,color='r', s=4)
plt.ylabel('BOLD (%)',color='r')
plt.tick_params(axis='y',labelcolor='r')
plt.xlabel('PETCO$_{2}$ (mmHg)')
plt.tick_params(axis='y',labelcolor='r')
plt.title('BOLD vs PETCO$_{2}$')

print(type(mrpetco2))
print(type(func))
print(type(petco2))
print(type(mca))

# plot func as a function of petco2 on the same plot with a different y-axis on the right

# TCD RAMP FITTING

In [ ]:
# Calculate the baseline values for mca and petco2
mca_base = cvr_func.get_baseline_values(mca, 17, 8)
petco2_base = cvr_func.get_baseline_values(petco2, 17, 8)

# mca_base = mca.iloc[0:5,1].mean()
# petco2_base = petco2.iloc[0:5,1].mean()
print('---BASELINE VALUES---')
print('MCAvmean baseline value (cm/s):', mca_base)
print('PETCO2 baseline value (mmHg)', petco2_base)

In [ ]:
# rename mca_ramp_up to mca, and petco2_ramp_up to petco2
mca = mca_ramp_up
petco2 = petco2_ramp_up

In [ ]:
# NORMALIZE THE MCA DATA RELATIVE TO THE BASELINE MCAvmean
mca_norm = copy.deepcopy(mca)
# scale the mca data (2nd column) in mca_norm by the mean of the lowest values
mca_norm.iloc[:,0] = mca_norm.iloc[:,0] / mca_base

print(mca.shape)
print(mca_norm.shape)

mca = mca_norm

In [ ]:
# plot X = PETCO2, Y = MCAvmean

# do a linear regression of the data
slope, intercept, r_value, p_value, std_err = stats.linregress(petco2.iloc[:,0],mca.iloc[:,0])
# print the equation of the line
print('y = ' + str(slope) + 'x + ' + str(intercept))
# print the p value
print('p = ' + str(p_value))
# print the r value
print('r = ' + str(r_value))

fig = plt.figure(figsize=(10,5))
plt.scatter(petco2.iloc[:,0],mca.iloc[:,0], s=2, c='k')
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('MCAvmean (normalised)')
plt.plot(petco2.iloc[:,0], slope*petco2.iloc[:,0] + intercept, 'r')
plt.title('y = ' + str(slope) + 'x + ' + str(intercept) + ', p = ' + str(round(p_value,4)))
plt.show()

print('LINEAR CVR = ' + str(slope*100) + ' %/mmHg')

In [ ]:
## MODELS ##

# fit a 4 parameter logistic curve to the data
# 4p model
def fit_4pl(x, a, b, c, d):
    y = a + (d / (1 + np.exp(-(x-c)/b)))
    return (y)
# print the equation and parameters
print('----- 4PL MODEL -----')
model_str_4p = 'a + (d / (1 + np.exp((c-x)/b)'
print(model_str_4p)
# where a = min, b = slope, c = inflection point, d = span
# x = petco2.iloc[:end_ramp1_index,0]
# y = mca.iloc[:end_ramp1_index,0]
x = petco2.iloc[:,0]
y = mca.iloc[:,0]
print(y.max())
#initial guess
a0 = 0.3
b0 = 2.5
c0 = 35
d0 = 2.5 
p0_4p = [a0, b0, c0, d0]
print('Initial guesses for a0, b0, c0, d0:',p0_4p)
#BOUNDS
p0_bounds_4p=([0, 0, 0, 1],[1, 15, 60, 4])
print('Bounds for a, b, c, d:',p0_bounds_4p)

#fit 4pl
p_opt_4p, cov_p_4p = curve_fit(fit_4pl, x, y, p0=p0_4p, bounds=p0_bounds_4p, maxfev=5000)
#get optimized model
a_opt_4p, b_opt_4p, c_opt_4p, d_opt_4p = p_opt_4p
x_model_4p = np.linspace(0, 80)
y_model_4p = fit_4pl(x_model_4p, a_opt_4p, b_opt_4p, c_opt_4p, d_opt_4p)

# fit a 5 parameter logistic curve to the data
# 5p MODEL
def fit_5pl(x, a, b, c, d, s):
    y = a + ((d) / (1 + np.exp((c-x)/b)))**s
    return (y)
# print the equation and parameters
print('----- 5PL MODEL -----')
model_str_5p = 'y = a + ((d) / (1 + np.exp((c-x)/b)))**s'
print(model_str_5p)
# where a = min, b = slope, c = inflection point, d = span, s = symmetry
# x = petco2.iloc[:end_ramp1_index,0]
# y = mca.iloc[:end_ramp1_index,0]
#initial guess
a0 = 0.3
b0 = 2.5
c0 = 35
d0 = 2.5 
s0 = 1
p0_5p = [a0, b0, c0, d0, s0]
print('Initial guesses for a0, b0, c0, d0, s0:',p0_5p)
#BOUNDS
p0_bounds_5p=([0.2, 0, 0, 1, 0],[1, 15, 60, 4, 20])
print('Bounds for a, b, c, d, s:',p0_bounds_5p)

#fit 5pl
p_opt_5p, cov_p_5p = curve_fit(fit_5pl, x, y, p0=p0_5p, bounds=p0_bounds_5p, maxfev=5000)
#get optimized model
a_opt_5p, b_opt_5p, c_opt_5p, d_opt_5p, s_opt_5p = p_opt_5p
x_model_5p = np.linspace(0, 80)
y_model_5p = fit_5pl(x_model_5p, a_opt_5p, b_opt_5p, c_opt_5p, d_opt_5p, s_opt_5p)


# fit a 2 parameter logistic curve to the data
# 2p model

def fit_2pl_tcd(x, b, c):
    a = 0.2
    d = 2.5
    y = a + (d / (1 + np.exp(-(x-c)/b)))
    return (y)

# print the equation and parameters
print('----- 2PL MODEL -----')
model_str_2p_tcd = '0.2 + (2.5 / (1 + np.exp(-(x-c)/b))'
print(model_str_2p_tcd)
# where b = slope, c = inflection point and the minimum and the maximum are fixed
# x = petco2.iloc[:end_ramp1_index,0]
# y = mca.iloc[:end_ramp1_index,0]
x = petco2.iloc[:,0]
y = mca.iloc[:,0]
print(y.max())
#initial guess
b0 = 30
c0 = 35
p0_2p_tcd = [b0, c0]
print('Initial guesses for b0, c0:',p0_2p_tcd)
#BOUNDS
p0_bounds_2p_tcd=([0, 0],[55, 70])
print('Bounds for b, c:',p0_bounds_2p_tcd)

#fit 2pl
p_opt_2p_tcd, cov_p_2p_tcd = curve_fit(fit_2pl_tcd, x, y, p0=p0_2p_tcd, bounds=p0_bounds_2p_tcd, maxfev=5000)
#get optimized model
b_opt_2p_tcd, c_opt_2p_tcd = p_opt_2p_tcd
x_model_2p_tcd = np.linspace(0, 80)
y_model_2p_tcd = fit_2pl_tcd(x_model_2p_tcd, b_opt_2p_tcd, c_opt_2p_tcd)


In [ ]:
# plot X = PETCO2, Y = MCAvmean Scatter Plot

fig_4p = plt.figure(figsize=(10,5))
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(petco2.iloc[:,0],mca.iloc[:,0], s=4, c='k')
sns.lineplot(x=x_model_4p, y=y_model_4p, color='r', alpha=0.5)
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('Normalised MCAvmean')
plt.ylim(0,2)
plt.xlim(0,80)

plt.title('SUBJECT: ' + sub + ' | '+ model_str_4p +' | 4PL')

# print parameters of the fit on the plot
plt.text(0.05,0.9,'a = ' + str(round(a_opt_4p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.85,'b = ' + str(round(b_opt_4p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.8,'c = ' + str(round(c_opt_4p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.75,'d = ' + str(round(d_opt_4p,2)),transform=plt.gca().transAxes)
# remove the legend
plt.legend().remove()

plt.show()

In [ ]:
fig_5p = plt.figure(figsize=(10,5))
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(petco2.iloc[:,0],mca.iloc[:,0], s=4, c='k')
sns.lineplot(x=x_model_5p, y=y_model_5p, color='r', alpha=0.5)
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('Normalised MCAvmean')
plt.ylim(0,2)
plt.xlim(0,80)

plt.title('SUBJECT: ' + sub + ' | '+ model_str_5p +' | 5PL')

# print parameters of the fit on the plot
plt.text(0.05,0.9,'a = ' + str(round(a_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.85,'b = ' + str(round(b_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.8,'c = ' + str(round(c_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.75,'d = ' + str(round(d_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.7,'s = ' + str(round(s_opt_5p,2)),transform=plt.gca().transAxes)
# remove the legend
plt.legend().remove()

plt.show()

print(petco2_ramp_up.shape)

In [ ]:
# plot X = PETCO2, Y = MCAvmean Scatter Plot

fig_2p_tcd = plt.figure(figsize=(10,5))
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(petco2.iloc[:,0],mca.iloc[:,0], s=4, c='k')
sns.lineplot(x=x_model_2p_tcd, y=y_model_2p_tcd, color='r', alpha=0.5)
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('Normalised MCAvmean')
plt.ylim(0,2)
plt.xlim(0,80)

plt.title('SUBJECT: ' + sub + ' | '+ model_str_2p_tcd +' | 2PL')

# print parameters of the fit on the plot
plt.text(0.05,0.9,'b = ' + str(round(b_opt_2p_tcd,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.85,'c = ' + str(round(c_opt_2p_tcd,2)),transform=plt.gca().transAxes)

# remove the legend
plt.legend().remove()

plt.show()

In [ ]:
# save the figure
if save_fig == True:    
    fig_4p.savefig(sub_outdir + sub + '_4pl.png')
    fig_5p.savefig(sub_outdir + sub + '_5pl.png')
    fig_2p_tcd.savefig(sub_outdir + sub + '_2pl_tcd.png')

In [ ]:
# information about the PETCO2 and MCAvmean data
print('TCD ramp information')
# get the mean of the lowest 3 values of the petco2 data
petco2_min = petco2.iloc[:,0].nsmallest(3)
# get the mean of petco2_min
petco2_min = petco2_min.mean()
# get the highest 3 values of the petco2 data
petco2_max = petco2.iloc[:,0].nlargest(3)
# get the mean of petco2_max
petco2_max = petco2_max.mean()


petco2_diff = petco2_max - petco2_min
print('PETCO2 max', petco2_max)
print('PETCO2 min', petco2_min)
print('PETCO2 diff', petco2_diff)

# get the mean of the lowest 3 values of the mca data
mca_min = mca.iloc[:,0].nsmallest(3)
# get the mean of mca_min
mca_min = mca_min.mean()
# get the highest 3 values of the mca data
mca_max = mca.iloc[:,0].nlargest(3)
# get the mean of mca_max
mca_max = mca_max.mean()

mca_diff = mca_max - mca_min
print('MCAvmean max', mca_max)
print('MCAvmean min', mca_min)
print('MCAvmean diff', mca_diff)

In [ ]:
# define the x and y values for the curve fitting
x = petco2.iloc[:,0]
y = mca.iloc[:,0]

# ensure they are the same length
print(x.shape)
print(y.shape)

In [ ]:
# define the 4p logistic model
model1 = PolynomialModel(0) + LogisticModel()

# minimum center at ~30% +/- 15% blood flow (limits at 0.15*8 = +/- 1.2)
model1.setPrior( 0, prior=GaussPrior(center=0.3, scale=0.15, limits=[0,1]))
# span center at ~250% +/- 80% blood flow (limits at 0.5*8 = +/- 4)
model1.setPrior( 1, prior=GaussPrior(center=2.5, scale=0.5, limits=[1,4]))
model1.setPrior( 2, prior=GaussPrior(center=35, scale=4, limits=[0,60]))
model1.setPrior( 3, prior=GaussPrior(center=2.5, scale=5, limits=[0,15]))

In [ ]:
# define NestedSampler
ns = NestedSampler( x, model1, y, weights=None, seed=1301 )
# set limits on the noise scale of the distribution
ns.distribution.setLimits( [0.01,100] )
# run NestedSampler
evi = ns.sample( plot=True )

print(model1.getPrior(0))
print(model1.getPrior(1))
print(model1.getPrior(2))
print(model1.getPrior(3))

In [ ]:
sl = ns.samples
par = sl.parameters
std = sl.stdevs
print( "             p1       p2       p3       p4    ")
print( "params  ", fmt( par, max=None ) )
print( "stdevs  ", fmt( std, max=None ) )
pal = par.copy()
stl = std.copy()
print( "params  ", fmt( pal, max=None ) )
print( "stdevs  ", fmt( stl, max=None ) )
print( "scale   ", fmt( sl.scale ), " +-", fmt( sl.stdevScale ) )
print( "evidence", fmt( evi ) )

In [ ]:
m1 = PowerLawModel( fixed={0:1.0, 1:0.0} )
m2 = LogisticModel()
m3 = m2 | m1
model5p = m3
print('--- before incorporating offset ---')
print( model5p )
print('Number of parameters in model5p:', model5p.getNumberOfParameters())

# span center at ~250% +/- 80% blood flow (limits at 0.5*8 = +/- 4)
model5p.setPrior( 0, prior=GaussPrior(center=2.5, scale=0.5, limits=[1,4]))
model5p.setPrior( 1, prior=GaussPrior(center=35,scale=4, limits=[0,60]))
model5p.setPrior( 2, prior=GaussPrior(center=2.5, scale=5, limits=[0,15]))
model5p.setPrior( 3, prior=GaussPrior(center=1, scale=0.5, limits=[0.1,10]))

# Add an offset to the logistic model
polynomial = PolynomialModel(0)
polynomial.setLimits( [0],[1] )
#polynomial.setPrior( 0, prior=GaussPrior(center=0.25, limits=[0.15,0.5]))
polynomial.setPrior( 0, prior=GaussPrior(center=0.3, scale=0.15, limits=[0,1]))

print('--- after incorporating offset ---')
model5p = polynomial + model5p
print( model5p )
print('Number of parameters in model5p:', model5p.getNumberOfParameters())

# define NestedSampler
ns5p = NestedSampler( x, model5p, y, weights=None, seed=1127 )
# set limits on the noise scale of the distribution
ns5p.distribution.setLimits( [0.01,100] )
# run NestedSampler
evi5p = ns5p.sample( plot=True )

print(model5p.getPrior(0))
print(model5p.getPrior(1))
print(model5p.getPrior(2))
print(model5p.getPrior(3))
print(model5p.getPrior(4))

In [ ]:
sl5p = ns5p.samples
par5p = sl5p.parameters
std5p = sl5p.stdevs
print( "                p0       p1       p2       p3       p4    ")
print( "params  ", fmt( par5p, max=None ) )
print( "stdevs  ", fmt( std5p, max=None ) )
pal5p = par5p.copy()
stl5p = std5p.copy()
print( "params  ", fmt( pal5p, max=None ) )
print( "stdevs  ", fmt( stl5p, max=None ) )
print( "scale   ", fmt( sl5p.scale ), " +-", fmt( sl5p.stdevScale ) )
print( "evidence", fmt( evi5p ) )

In [ ]:
fig_fits_tcd = plt.figure( "nestedsamplerfit", figsize=[10,5] )
# increase font size of the axis labels on the plots
plt.rcParams.update({'font.size': 14})

print('size of PETCO2:', x.shape)
print('size of MCAvmean:', y.shape)
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'


xx = np.linspace( 0, 75, 100, dtype=float )

plt.plot(xx, slope*xx + intercept, 'g-')

plt.plot( xx, model1.result( xx, par ), 'b-')
#plt.plot( xx, model1.result( xx, param2 ), 'r-' )
plt.plot( xx, model5p.result( xx, par5p ), 'r-' )

plt.plot( x_model_4p, y_model_4p, 'b-.' )
plt.plot( x_model_5p, y_model_5p, 'r-.' )


plt.plot( x, y, 'k. ', markersize=2 )


# sns.lineplot(x=x_model_4p, y=y_model_4p, color='y', alpha=0.5)
# sns.lineplot(x=x_model_5p, y=y_model_5p, color='b', alpha=0.5)
plt.xlabel( "PETCO$_{2}$ (mmHg)")
plt.ylabel( "MCAv$_{mean}$ (normalised)")
plt.title('MCAvmean vs PETCO$_{2}$ (' + sub + ')')

plt.legend( ["Linear Fit", "4p Bayes Fit", "5p Bayes Fit", "4p LSR Fit", "5p LSR Fit", "Data"])
#plt.legend().remove()
plt.xlim( 0, 75 )
plt.ylim( 0, 2)
plt.show()

In [ ]:
if save_fig == True:    
    fig_fits_tcd.savefig(sub_outdir + sub + '_linefits_tcd.png')

In [ ]:
# calculate a shift parameter

# define the shift parameter as the 'c' parameter minus the petco2_base
shift_par_4p_LSR = petco2_base - c_opt_4p
shift_par_5p_LSR = petco2_base - c_opt_5p

shift_par_4p_bayes = petco2_base - par[2]
shift_par_5p_bayes =  petco2_base - par5p[2]

print('---SHIFT PARAMETERS---')
print('4PL LSR shift parameter:', shift_par_4p_LSR)
print('5PL LSR shift parameter:', shift_par_5p_LSR)
print('4PL Bayes shift parameter:', shift_par_4p_bayes)
print('5PL Bayes shift parameter:', shift_par_5p_bayes)

In [ ]:
# save a log file with the values of the PARAMETERS for the LSR and Bayesian fits (4p and 5p)

if save_log == True:

    print('Subject TCD:', sub_tcd, 'MRI:', sub)

    csv_para_log_name_tcd_bayes = outdir_logs + 'parameter_log_tcd_bayes.csv'
    para_log_tcd_bayes = pd.DataFrame({'sub':[sub_tcd],'mr-sub':[sub],'a_opt_4p_bayes':[par[0]], 'b_opt_4p_bayes':[par[3]], 'c_opt_4p_bayes':[par[2]], 'd_opt_4p_bayes':[par[1]], 'shift_par_4p_bayes':[shift_par_4p_bayes], 'a_opt_5p_bayes':[par5p[0]], 'b_opt_5p_bayes':[par5p[3]], 'c_opt_5p_bayes':[par5p[2]], 'd_opt_5p_bayes':[par5p[1]], 's_opt_5p_bayes':[par5p[4]], 'shift_par_5p_bayes':[shift_par_5p_bayes]})
    # if the file does not exist, create it
    if not os.path.exists(csv_para_log_name_tcd_bayes):
        para_log_tcd_bayes.to_csv(csv_para_log_name_tcd_bayes, index=False)
        print('Bayes parameter log saved to new file:', csv_para_log_name_tcd_bayes)
    # if the file exists, append to it
    else:
        para_log_tcd_bayes.to_csv(csv_para_log_name_tcd_bayes, mode='a', header=False, index=False)
        print('Bayes parameter log appended to existing file:', csv_para_log_name_tcd_bayes)


    csv_para_log_name_tcd_LSR = outdir_logs + 'parameter_log_tcd_LSR.csv'
    para_log_tcd_LSR = pd.DataFrame({'sub':[sub_tcd],'mr-sub':[sub], 'slope_lin':[slope], 'intercept_lin':[intercept], 'a_opt_4p':[a_opt_4p], 'b_opt_4p':[b_opt_4p], 'c_opt_4p':[c_opt_4p], 'd_opt_4p':[d_opt_4p], 'shift_par_4p_LSR':[shift_par_4p_LSR], 'a_opt_5p':[a_opt_5p], 'b_opt_5p':[b_opt_5p], 'c_opt_5p':[c_opt_5p], 'd_opt_5p':[d_opt_5p], 's_opt_5p':[s_opt_5p], 'shift_par_5p_LSR':[shift_par_5p_LSR]})
    # if the file does not exist, create it
    if not os.path.exists(csv_para_log_name_tcd_LSR):
        para_log_tcd_LSR.to_csv(csv_para_log_name_tcd_LSR, index=False)
        print('LSR parameter log saved to new file:', csv_para_log_name_tcd_LSR)
    # if the file exists, append to it
    else:
        para_log_tcd_LSR.to_csv(csv_para_log_name_tcd_LSR, mode='a', header=False, index=False)
        print('LSR parameter log appended to existing file:', csv_para_log_name_tcd_LSR)

In [ ]:
# save and append the max, min, and diff of the petco2 and mca data to a csv file if save_log is true for each subject

tcd_log = outdir_logs + 'tcd_log.csv'

if save_log == True:
    # get the petco2 max
    petco2_max = max(petco2.iloc[:,0])
    # get the petco2 min
    petco2_min = min(petco2.iloc[:,0])
    # get the petco2 diff
    petco2_diff = petco2_max - petco2_min
    # get the mca max
    mca_max = max(mca.iloc[:,0])
    # get the mca min
    mca_min = min(mca.iloc[:,0])
    # get the mca diff
    mca_diff = mca_max - mca_min
    # get the petco2 diff
    petco2_diff = petco2_diff
    # get the mca diff
    mca_diff = mca_diff

    # create the csv file if it doesn't exist, otherwise append to it
    if not os.path.exists(tcd_log):
        with open(tcd_log, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["sub", "mr-sub", "petco2_max","petco2_min","petco2_diff","mca_max","mca_min","mca_diff","petco2_base","mca_base"])
            writer.writerow([sub_tcd,sub,petco2_max,petco2_min,petco2_diff,mca_max,mca_min,mca_diff,petco2_base,mca_base])
    else:
        with open(tcd_log, 'a') as f:
            writer = csv.writer(f)
            writer.writerow([sub_tcd,sub,petco2_max,petco2_min,petco2_diff,mca_max,mca_min,mca_diff,petco2_base,mca_base])

# MRI RAMP FITTING

In [ ]:
# rename mca_ramp_up to mca, and petco2_ramp_up to petco2
func = func_ramp_up
mrpetco2 = mrpetco2_ramp_up

In [ ]:
# plot X = MRPETCO2, Y = FUNC

# do a linear regression of the data
slope, intercept, r_value, p_value, std_err = stats.linregress(mrpetco2.iloc[:,0],func.iloc[:,0])
# print the equation of the line
print('y = ' + str(slope) + 'x + ' + str(intercept))
# print the p value
print('p = ' + str(p_value))
# print the r value
print('r = ' + str(r_value))

fig = plt.figure(figsize=(10,5))
plt.scatter(mrpetco2.iloc[:,0],func.iloc[:,0], s=2, c='k')
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('BOLD signal (normalised)')
plt.plot(mrpetco2.iloc[:,0], slope*mrpetco2.iloc[:,0] + intercept, 'r')
plt.title('y = ' + str(slope) + 'x + ' + str(intercept) + ', p = ' + str(round(p_value,4)))
plt.show()

print('LINEAR CVR = ' + str(slope*100) + ' %/mmHg')

In [ ]:
## MODELS ##

# fit a 4 parameter logistic curve to the data
# 4p model
def fit_4pl(x, a, b, c, d):
    y = a + (d / (1 + np.exp(-(x-c)/b)))
    return (y)
# print the equation and parameters
print('----- 4PL MODEL -----')
model_str_4p = 'a + (d / (1 + np.exp((c-x)/b)'
print(model_str_4p)
# where a = min, b = slope, c = inflection point, d = span
# x = mrpetco2.iloc[:end_ramp1_index,0]
# y = func.iloc[:end_ramp1_index,0]
x = mrpetco2.iloc[:,0]
y = func.iloc[:,0]
print(y.max())
#initial guess
a0 = -8
b0 = 10
c0 = 35
d0 = 18
p0_4p = [a0, b0, c0, d0]
print('Initial guesses for a0, b0, c0, d0:',p0_4p)
#BOUNDS
p0_bounds_4p=([-20, 0, 0, 1],[0, 25, 60, 40])
print('Bounds for a, b, c, d:',p0_bounds_4p)

#fit 4pl
p_opt_4p, cov_p_4p = curve_fit(fit_4pl, x, y, p0=p0_4p, bounds=p0_bounds_4p, maxfev=5000)
#get optimized model
a_opt_4p, b_opt_4p, c_opt_4p, d_opt_4p = p_opt_4p
x_model_4p = np.linspace(0, 80)
y_model_4p = fit_4pl(x_model_4p, a_opt_4p, b_opt_4p, c_opt_4p, d_opt_4p)


# fit a 5 parameter logistic curve to the data
# 5p MODEL
def fit_5pl(x, a, b, c, d, s):
    y = a + ((d) / (1 + np.exp((c-x)/b)))**s
    return (y)
# print the equation and parameters
print('----- 5PL MODEL -----')
model_str_5p = 'y = a + ((d) / (1 + np.exp((c-x)/b)))**s'
print(model_str_5p)
# where a = min, b = slope, c = inflection point, d = span, s = symmetry
# x = mrpetco2.iloc[:end_ramp1_index,0]
# y = func.iloc[:end_ramp1_index,0]
#initial guess
a0 = -8
b0 = 10
c0 = 35
d0 = 18
s0 = 1
p0_5p = [a0, b0, c0, d0, s0]
print('Initial guesses for a0, b0, c0, d0, s0:',p0_5p)
#BOUNDS
p0_bounds_5p=([-20, 0, 0, 1, 0],[0, 25, 60, 40, 10])
print('Bounds for a, b, c, d, s:',p0_bounds_5p)

#fit 5pl
p_opt_5p, cov_p_5p = curve_fit(fit_5pl, x, y, p0=p0_5p, bounds=p0_bounds_5p, maxfev=3000)
#get optimized model
a_opt_5p, b_opt_5p, c_opt_5p, d_opt_5p, s_opt_5p = p_opt_5p
x_model_5p = np.linspace(0, 80)
y_model_5p = fit_5pl(x_model_5p, a_opt_5p, b_opt_5p, c_opt_5p, d_opt_5p, s_opt_5p)

# fit a 2 parameter logistic curve to the data
# 2p model

def fit_2pl_mri(x, b, c):
    a = -8
    d = 18
    y = a + (d / (1 + np.exp(-(x-c)/b)))
    return (y)
# print the equation and parameters
print('----- 2PL MODEL -----')
model_str_2p_mri = '-8 + (18 / (1 + np.exp(-(x-c)/b))'
print(model_str_2p_mri)

# where b = slope, c = inflection point and the minimum and the maximum are fixed
# x = mrpetco2.iloc[:end_ramp1_index,0]
# y = func.iloc[:end_ramp1_index,0]
x = mrpetco2.iloc[:,0]
y = func.iloc[:,0]
print(y.max())
#initial guess
b0 = 8
c0 = 35
p0_2p_mri = [b0, c0]
print('Initial guesses for b0, c0:',p0_2p_mri)
#BOUNDS
p0_bounds_2p_mri=([0, 0],[25, 60])
print('Bounds for b, c:',p0_bounds_2p_mri)

#fit 2pl
p_opt_2p_mri, cov_p_2p_mri = curve_fit(fit_2pl_mri, x, y, p0=p0_2p_mri, bounds=p0_bounds_2p_mri)
#get optimized model
b_opt_2p_mri, c_opt_2p_mri = p_opt_2p_mri
x_model_2p_mri = np.linspace(0, 80)
y_model_2p_mri = fit_2pl_mri(x_model_2p_mri, b_opt_2p_mri, c_opt_2p_mri)

In [ ]:
# plot X = MRPETCO2, Y = FUNC Scatter Plot

fig_4p = plt.figure(figsize=(10,5))
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(mrpetco2.iloc[:,0],func.iloc[:,0], s=4, c='k')
sns.lineplot(x=x_model_4p, y=y_model_4p, color='r', alpha=0.5)
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('Normalised BOLD signal')
plt.ylim(-6,6)
plt.xlim(0,80)

plt.title('SUBJECT: ' + sub + ' | '+ model_str_4p +' | 4PL')

# print parameters of the fit on the plot
plt.text(0.05,0.9,'a = ' + str(round(a_opt_4p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.85,'b = ' + str(round(b_opt_4p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.8,'c = ' + str(round(c_opt_4p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.75,'d = ' + str(round(d_opt_4p,2)),transform=plt.gca().transAxes)
# remove the legend
plt.legend().remove()

plt.show()

In [ ]:
fig_5p = plt.figure(figsize=(10,5))
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.scatter(mrpetco2.iloc[:,0],func.iloc[:,0], s=4, c='k')
sns.lineplot(x=x_model_5p, y=y_model_5p, color='r', alpha=0.5)
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('Normalised BOLD signal')
plt.ylim(-6,6)
plt.xlim(0,80)

plt.title('SUBJECT: ' + sub + ' | '+ model_str_5p +' | 5PL')

# print parameters of the fit on the plot
plt.text(0.05,0.9,'a = ' + str(round(a_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.85,'b = ' + str(round(b_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.8,'c = ' + str(round(c_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.75,'d = ' + str(round(d_opt_5p,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.7,'s = ' + str(round(s_opt_5p,2)),transform=plt.gca().transAxes)
# remove the legend
plt.legend().remove()

plt.show()

In [ ]:
# plot the 2p model
fig_2p_mri = plt.figure(figsize=(10,5))

plt.scatter(mrpetco2.iloc[:,0],func.iloc[:,0], s=4, c='k')
sns.lineplot(x=x_model_2p_mri, y=y_model_2p_mri, color='r', alpha=0.5)
plt.xlabel('P$_{ET}$CO$_{2}$ (mmHg)')
plt.ylabel('Normalised BOLD signal')
plt.ylim(-6,6)
plt.xlim(0,80)

plt.title('SUBJECT: ' + sub + ' | '+ model_str_2p_mri +' | 2PL')

# print parameters of the fit on the plot
plt.text(0.05,0.85,'b = ' + str(round(b_opt_2p_mri,2)),transform=plt.gca().transAxes)
plt.text(0.05,0.8,'c = ' + str(round(c_opt_2p_mri,2)),transform=plt.gca().transAxes)

# remove the legend
plt.legend().remove()

plt.show()


In [ ]:
# save the figure
if save_fig == True:
    fig_4p.savefig(sub_outdir + sub + '_linefits_mri_4p.png')
    fig_5p.savefig(sub_outdir + sub + '_linefits_mri_5p.png')
    fig_2p_mri.savefig(sub_outdir + sub + '_linefits_mri_2p.png')

In [ ]:
# define the shift parameter as the 'c' parameter minus the petco2_base
shift_par_2p_tcd = petco2_base - c_opt_2p_tcd
shift_par_2p_mri = petco2_base - c_opt_2p_mri

In [ ]:
# save the 2p mri and tcd parameters (b and c) to a csv file

if save_log == True:
    if not os.path.exists(outdir_logs + '2p_fits_LSR.csv'):
        with open(outdir_logs + '2p_fits_LSR.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["sub", "mr-sub", "b_2p_opt_tcd", "c_2p_opt_tcd", "shift_par_2p_tcd","b_2p_opt_mri", "c_2p_opt_mri", "shift_par_2p_mri"])
            writer.writerow([sub_tcd, sub, b_opt_2p_tcd, c_opt_2p_tcd, shift_par_2p_tcd, b_opt_2p_mri, c_opt_2p_mri, shift_par_2p_mri])


    else:
        with open(outdir_logs + '2p_fits_LSR.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([sub_tcd, sub, b_opt_2p_tcd, c_opt_2p_tcd, shift_par_2p_tcd, b_opt_2p_mri, c_opt_2p_mri, shift_par_2p_mri])

In [ ]:
# define the x and y values for the curve fitting
x = mrpetco2.iloc[:,0]
y = func.iloc[:,0]

# ensure they are the same length
print(x.shape)
print(y.shape)

In [ ]:
# define the 4p logistic model
model1 = PolynomialModel(0) + LogisticModel()

# minimum center at ~30% +/- 15% blood flow (limits at 0.15*8 = +/- 1.2)
model1.setPrior( 0, prior=GaussPrior(center=-8, scale=0.15, limits=[-20,0])) #a
# span center at ~250% +/- 80% blood flow (limits at 0.5*8 = +/- 4)
model1.setPrior( 1, prior=GaussPrior(center=18, scale=0.5, limits=[1,40])) #d
model1.setPrior( 2, prior=GaussPrior(center=35, scale=4, limits=[0,60])) #c
model1.setPrior( 3, prior=GaussPrior(center=10, scale=5, limits=[0,25])) #b

In [ ]:
# define NestedSampler
ns = NestedSampler( x, model1, y, weights=None, seed=1301 )
# set limits on the noise scale of the distribution
ns.distribution.setLimits( [0.01,100] )
# run NestedSampler
evi = ns.sample( plot=True )

print(model1.getPrior(0))
print(model1.getPrior(1))
print(model1.getPrior(2))
print(model1.getPrior(3))

In [ ]:
sl = ns.samples
par = sl.parameters
std = sl.stdevs
print( "             p1       p2       p3       p4    ")
print( "params  ", fmt( par, max=None ) )
print( "stdevs  ", fmt( std, max=None ) )
pal = par.copy()
stl = std.copy()
print( "params  ", fmt( pal, max=None ) )
print( "stdevs  ", fmt( stl, max=None ) )
print( "scale   ", fmt( sl.scale ), " +-", fmt( sl.stdevScale ) )
print( "evidence", fmt( evi ) )

In [ ]:
m1 = PowerLawModel( fixed={0:1.0, 1:0.0} )
m2 = LogisticModel()
m3 = m2 | m1
model5p = m3
print('--- before incorporating offset ---')
print( model5p )
print('Number of parameters in model5p:', model5p.getNumberOfParameters())

# span center at ~250% +/- 80% blood flow (limits at 0.5*8 = +/- 4)
model5p.setPrior( 0, prior=GaussPrior(center=18, scale=0.5, limits=[1,40])) #d
model5p.setPrior( 1, prior=GaussPrior(center=35,scale=4, limits=[0,60])) #c
model5p.setPrior( 2, prior=GaussPrior(center=10, scale=5, limits=[0,25])) #b
model5p.setPrior( 3, prior=GaussPrior(center=1, scale=0.5, limits=[0.1,10])) #s

# Add an offset to the logistic model
polynomial = PolynomialModel(0)
polynomial.setLimits( [0],[1] )
#polynomial.setPrior( 0, prior=GaussPrior(center=0.25, limits=[0.15,0.5]))
polynomial.setPrior( 0, prior=GaussPrior(center=-8, scale=0.15, limits=[-20,0])) #a

print('--- after incorporating offset ---')
model5p = polynomial + model5p
print( model5p )
print('Number of parameters in model5p:', model5p.getNumberOfParameters())

# define NestedSampler
ns5p = NestedSampler( x, model5p, y, weights=None, seed=1127 )
# set limits on the noise scale of the distribution
ns5p.distribution.setLimits( [0.01,100] )
# run NestedSampler
evi5p = ns5p.sample( plot=True )

print(model5p.getPrior(0))
print(model5p.getPrior(1))
print(model5p.getPrior(2))
print(model5p.getPrior(3))
print(model5p.getPrior(4))

In [ ]:
sl5p = ns5p.samples
par5p = sl5p.parameters
std5p = sl5p.stdevs
print( "                p0       p1       p2       p3       p4    ")
print( "params  ", fmt( par5p, max=None ) )
print( "stdevs  ", fmt( std5p, max=None ) )
pal5p = par5p.copy()
stl5p = std5p.copy()
print( "params  ", fmt( pal5p, max=None ) )
print( "stdevs  ", fmt( stl5p, max=None ) )
print( "scale   ", fmt( sl5p.scale ), " +-", fmt( sl5p.stdevScale ) )
print( "evidence", fmt( evi5p ) )

In [ ]:
fig_fits_mri = plt.figure( "nestedsamplerfit", figsize=[10,5] )
# increase font size of the axis labels on the plots
plt.rcParams.update({'font.size': 14})

print('size of MRPETCO2:', x.shape)
print('size of FUNC (normalised):', y.shape)
# set font to times new roman
plt.rcParams['font.family'] = 'Times New Roman'

xx = np.linspace( 0, 75, 100, dtype=float )

plt.plot(xx, slope*xx + intercept, 'g-')

plt.plot( xx, model1.result( xx, par ), 'b-')
plt.plot( xx, model5p.result( xx, par5p ), 'r-' )

plt.plot( x_model_4p, y_model_4p, 'b-.' )
plt.plot( x_model_5p, y_model_5p, 'r-.' )

plt.plot( x, y, 'k. ', markersize=2 )


# sns.lineplot(x=x_model_4p, y=y_model_4p, color='y', alpha=0.5)
# sns.lineplot(x=x_model_5p, y=y_model_5p, color='b', alpha=0.5)
plt.xlabel( "PETCO$_{2}$ (mmHg)")
plt.ylabel( "BOLD (%)")
plt.title('BOLD vs PETCO$_{2}$ (' + sub + ')')

plt.legend( ["Linear Fit", "4p Bayes Fit", "5p Bayes Fit", "4p LSR Fit", "5p LSR Fit", "Data"])
#plt.legend().remove()
plt.xlim( 0, 75 )
plt.ylim( -7, 7)
plt.show()

In [ ]:
if save_fig == True:    
    fig_fits_mri.savefig(sub_outdir + sub + '_linefits_mri.png')

In [ ]:
# calculate a shift parameter

# define the shift parameter as the 'c' parameter minus the petco2_base
mrshift_par_4p_LSR = mrpetco2_base - c_opt_4p
mrshift_par_5p_LSR = mrpetco2_base - c_opt_5p

mrshift_par_4p_bayes = mrpetco2_base - par[2]
mrshift_par_5p_bayes =  mrpetco2_base - par5p[2]

print('---SHIFT PARAMETERS---')
print('4PL LSR shift parameter:', mrshift_par_4p_LSR)
print('5PL LSR shift parameter:', mrshift_par_5p_LSR)
print('4PL Bayes shift parameter:', mrshift_par_4p_bayes)
print('5PL Bayes shift parameter:', mrshift_par_5p_bayes)

In [ ]:
# save a log file with the values of the PARAMETERS for the LSR and Bayesian fits (4p and 5p)

if save_log == True:

    print('Subject TCD:', sub_tcd, 'MRI:', sub)

    csv_para_log_name_mri_bayes = outdir_logs + 'parameter_log_mri_bayes.csv'
    para_log_mri_bayes = pd.DataFrame({'sub':[sub_tcd],'mr-sub':[sub],'a_opt_4p_bayes':[par[0]], 'b_opt_4p_bayes':[par[3]], 'c_opt_4p_bayes':[par[2]], 'd_opt_4p_bayes':[par[1]], 'mrshift_par_4p_bayes':[mrshift_par_4p_bayes],'a_opt_5p_bayes':[par5p[0]], 'b_opt_5p_bayes':[par5p[3]], 'c_opt_5p_bayes':[par5p[2]], 'd_opt_5p_bayes':[par5p[1]], 's_opt_5p_bayes':[par5p[4]], 'mrshift_par_5p_bayes':[mrshift_par_5p_bayes]})
    # if the file does not exist, create it
    if not os.path.exists(csv_para_log_name_mri_bayes):
        para_log_mri_bayes.to_csv(csv_para_log_name_mri_bayes, index=False)
        print('Bayes parameter log saved to new file:', csv_para_log_name_mri_bayes)
    # if the file exists, append to it
    else:
        para_log_mri_bayes.to_csv(csv_para_log_name_mri_bayes, mode='a', header=False, index=False)
        print('Bayes parameter log appended to existing file:', csv_para_log_name_mri_bayes)


    csv_para_log_name_mri_LSR = outdir_logs + 'parameter_log_mri_LSR.csv'
    para_log_mri_LSR = pd.DataFrame({'sub':[sub_tcd], 'mr-sub':[sub], 'slope_lin':[slope], 'intercept_lin':[intercept], 'a_opt_4p':[a_opt_4p], 'b_opt_4p':[b_opt_4p], 'c_opt_4p':[c_opt_4p], 'd_opt_4p':[d_opt_4p], 'mrshift_par_4p_LSR':[mrshift_par_4p_LSR], 'a_opt_5p':[a_opt_5p], 'b_opt_5p':[b_opt_5p], 'c_opt_5p':[c_opt_5p], 'd_opt_5p':[d_opt_5p], 's_opt_5p':[s_opt_5p], 'mrshift_par_5p_LSR':[mrshift_par_5p_LSR]})
    # if the file does not exist, create it
    if not os.path.exists(csv_para_log_name_mri_LSR):
        para_log_mri_LSR.to_csv(csv_para_log_name_mri_LSR, index=False)
        print('LSR parameter log saved to new file:', csv_para_log_name_mri_LSR)
    # if the file exists, append to it
    else:
        para_log_mri_LSR.to_csv(csv_para_log_name_mri_LSR, mode='a', header=False, index=False)
        print('LSR parameter log appended to existing file:', csv_para_log_name_mri_LSR)

In [ ]:
# save and append the max, min, and diff of the mrpetco2 and bold func data to a csv file if save_log is true for each subject

mri_log = outdir_logs + 'mri_log.csv'

if save_log == True:
    # get the mrpetco2 max
    mrpetco2_max = max(mrpetco2.iloc[:,0])
    # get the mrpetco2 min
    mrpetco2_min = min(mrpetco2.iloc[:,0])
    # get the mrpetco2 diff
    mrpetco2_diff = mrpetco2_max - mrpetco2_min
    # get the func max
    func_max = max(func.iloc[:,0])
    # get the func min
    func_min = min(func.iloc[:,0])
    # get the func diff
    func_diff = func_max - func_min

    # create the csv file if it doesn't exist, otherwise append to it
    if not os.path.exists(mri_log):
        with open(mri_log, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["sub", "mr-sub", "mrpetco2_max","mrpetco2_min","mrpetco2_diff","func_max","func_min","func_diff","mrpetco2_base","func_mean"])
            writer.writerow([sub_tcd,sub,mrpetco2_max,mrpetco2_min,mrpetco2_diff,func_max,func_min,func_diff,mrpetco2_base,func_mean[0]])
    else:
        with open(mri_log, 'a') as f:
            writer = csv.writer(f)
            writer.writerow([sub_tcd,sub,mrpetco2_max,mrpetco2_min,mrpetco2_diff,func_max,func_min,func_diff,mrpetco2_base,func_mean[0]])

# Cropped sigmoid region analysis

In [ ]:
# crop the sigmoid data by removing all data points where the petco2 is less than petco2_base and where the petco2 is greater than petco2_base+14mmHg

# make a new dataframe and remove the data points where the petco2 is less than petco2_base
mri_cropped = mrpetco2.copy()
mri_cropped['func'] = func
mri_cropped = mri_cropped[mri_cropped.iloc[:,0] > mrpetco2_base-5]
mri_cropped = mri_cropped[mri_cropped.iloc[:,0] < mrpetco2_base-5 + 20]

# fit a straight line to the new data
slope_cropped, intercept_cropped, r_value_cropped, p_value_cropped, std_err_cropped = stats.linregress(mri_cropped.iloc[:,0],mri_cropped.iloc[:,1])

# plot the sigmoid data
fig_sigmoid = plt.figure(figsize=(10,5))
plt.plot( x, y, 'k. ', markersize=2 )
plt.scatter(mri_cropped.iloc[:,0],mri_cropped.iloc[:,1], s=7, c='r')
plt.plot(mri_cropped.iloc[:,0], slope_cropped*mri_cropped.iloc[:,0] + intercept_cropped, 'r')
# plot the slope on the plot
plt.text(0.05,0.9,'slope = ' + str(round(slope_cropped,2)),transform=plt.gca().transAxes)
plt.xlabel('P$_{ET}$CO$_2$ (mmHg)')
plt.ylabel('BOLD (%)')
plt.title('Sigmoid Data')

plt.show()

In [ ]:
# crop the tcd data by removing all data points where the petco2 is less than petco2_base and where the petco2 is greater than petco2_base+14mmHg

# make a new dataframe and remove the data points where the petco2 is less than petco2_base
tcd_cropped = petco2.copy()
tcd_cropped['mca'] = mca
tcd_cropped = tcd_cropped[tcd_cropped.iloc[:,0] > petco2_base-5]
tcd_cropped = tcd_cropped[tcd_cropped.iloc[:,0] < petco2_base-5 + 20]

# fit a straight line to the new data
slope_cropped_tcd, intercept_cropped_tcd, r_value_cropped_tcd, p_value_cropped_tcd, std_err_cropped_tcd = stats.linregress(tcd_cropped.iloc[:,0],tcd_cropped.iloc[:,1])

# plot the sigmoid data
fig_sigmoid_tcd = plt.figure(figsize=(10,5))
plt.scatter(petco2.iloc[:,0],mca.iloc[:,0], s=4, c='k')
plt.scatter(tcd_cropped.iloc[:,0],tcd_cropped.iloc[:,1], s=5, c='r')
plt.plot(tcd_cropped.iloc[:,0], slope_cropped_tcd*tcd_cropped.iloc[:,0] + intercept_cropped_tcd, 'r')
# plot the slope on the plot
plt.text(0.05,0.9,'slope = ' + str(round(slope_cropped_tcd,2)),transform=plt.gca().transAxes)
plt.xlabel('P$_{ET}$CO$_2$ (mmHg)')
plt.ylabel('MCAv$_{mean}$ (normalised)')
plt.title('Sigmoid Data')

plt.show()

In [ ]:
# save the cropped slope parameters to a csv file
# only save the sub, mr-sub, slope_cropped, and slope_cropped_tcd

if save_log == True:

    if not os.path.exists(outdir_logs + 'cropped_slopes_LR.csv'):
        with open(outdir_logs + 'cropped_slopes_LR.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["sub", "mr-sub", "slope_cropped", "slope_cropped_tcd", ])
            writer.writerow([sub_tcd, sub, slope_cropped, slope_cropped_tcd])


    else:
        with open(outdir_logs + 'cropped_slopes_LR.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([sub_tcd, sub, slope_cropped, slope_cropped_tcd])